In [7]:
import os
import requests
from lxml import etree
import pandas as pd
import re
import json
from selenium import webdriver
import time
import datetime


In [5]:
import config
import public_fun
def handle(search_word, page, s_date):
    url = 'https://api.quzili.cn/search2?tags={}&title=title&skip={}&size=50&time=99999&minpagenum=0&maxpagenum=99999'.format(search_word, page*50)
    # index_url2 = 'http://www.bailuzhiku.com{}'
    headers = {
        'Authorization': 'hmac id="AKIDdlutrcn7F4j62Fskwqbiqrki3q3j40r1vjjw", algorithm="hmac-sha1", headers="x-date", signature="BmEXoJSvY8trUMj3BDcqiYo+KhE="',
        'X-Date': 'Tue, 29 Sep 2020 07:00:50 GMT',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36',
    }
    res = requests.get(url=url, headers=headers)
    res.encoding = res.apparent_encoding
    data = json.loads(res.text)
    print(data)
    result = []
    if data['hits']:
        pages = page+1 if data['hist']['total'] > page*50 else 0
        for d in data['hits']['hits']:
            if public_fun.calc_date(s_date=s_date, e_date=d['_source']['time']):
                result.append(d['uuid'])
        return result, pages
    else:
        print('url1 404错误:', res.status_code, url)
        return [],0

def get_data(url, search_word, max_text):
    json_result = {'source': 'blzk', 'doc_id': '', 'date': '', 'download_url': '',
                   'org_name': '', 'page_num': '1', 'doc_type': 'NEW', 'title': ''}
    path = os.path.join(config.SAVE_PATH, search_word, 'news', 'bailuzhiku')
    res = requests.get(url=url, headers=config.HEADERS)
    html = etree.HTML(res.text)
    content_text = public_fun.filter_space_json(html.xpath('//*[@class="textarea-box policy-textarea"]//text()'))
    if len(content_text) > max_text:
        try:
            doc_id = url.split('/')[-1].replace('.html', '') # 文章ID
            title = html.xpath('//*[@class="policy-wrap-title"]/h1')[0]   #文章标题
            # author = html.xpath('//*[@class="left-details-head"]')[0]
            # summary = html.xpath('//*[@class="summary"]')[0]
            description = html.xpath('//*[@class="textarea-box policy-textarea"]')[0]   #文章内容
            html_list = [title, description]
            html_result = public_fun.filter_space_html(html_list)
            json_result['doc_id'] = doc_id
            json_result['date'] = public_fun.filter_space_json(html.xpath('//*[@class="attribute-table"]/tbody/tr[2]/td[1]/div/text()')[0]
                                                               .replace('-', ''))
            json_result['download_url'] = 'http://www.bailuzhiku.com/'+html.xpath('//*[@class="header-tool-link download"]/@href')[0]
            try:
                json_result['org_name'] = public_fun.filter_space_json(html.xpath('//*[@class="attribute-table"]/tbody/tr[1]/td[1]/div/text()')[0])
            except:
                json_result['org_name'] = ''
            json_result['title'] = public_fun.filter_space_json(html.xpath('//h1[@class="title"]/text()')[0])
            public_fun.write_down_json(path=path, filename=doc_id + '.json', text=json_result)
            public_fun.write_down_html(path=path, filename=doc_id + '.html', text=html_result)
        except Exception as e:
            print(e.__traceback__.tb_lineno, url, e)
    else:
        print('文章字数不足', max_text)

def main(search_word, max_art, max_text, s_date):
    '''
    铅笔道爬虫入口函数
    :param search_word:搜索关键词 
    :return: None
    '''
    page = 1
    url2_list = []
    while page:
        one_page_url,pages = handle(search_word=search_word, page=page, s_date=s_date)
        url2_list += one_page_url
        if len(url2_list) < max_art and page < int(pages):
            page += 1
        else:
            page = 0
    return url2_list
    # url2_list = url2_list[:5] if config.DEBUG else url2_list
    # for url2 in url2_list:
    #     get_data(url=url2, search_word=search_word, max_text=max_text)

In [6]:
p0 = '人工智能'
p1 = 100
p2 = 500
p3 = '2018-01-01'
r2 = main(search_word=p0, max_art=p1, max_text=p2, s_date=p3)



{'message': 'HMAC signature cannot be verified, the x-date header is out of date for HMAC Authentication'}


KeyError: 'hits'

In [15]:

# 请求测试部分
test_url = 'https://www.1c9u.com/prod-api/api/1c9u/documentInfo/list'
headers = {
    'Cookie': 'sidebarStatus=0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36',
}
form_data = json.loads('{"searchType":"word","word":"人工智能","recommend":"","fileType":"","type":"DOCUMENT","tag":"","useWildcard":1,"page":{"current":1,"size":10,"ascs":[],"descs":[],"sort":"+id"}}')
test_res = requests.post(url=test_url, headers=headers, data=form_data)
test_res.encoding = test_res.apparent_encoding
print(test_res.text)
data = json.loads(test_res.text)
# test_string = '云南省人民政府关于印'
# print(test_string in test_res.text)
# with open('test_check', 'w', encoding='utf-8') as f:
#     f.write(test_res.text)

{"status":500,"message":"参数解析异常"}


In [13]:
# 其他测试部分
# a = '{"searchType":"word","word":"人工智能","recommend":"","fileType":"","type":"DOCUMENT","tag":"","useWildcard":1,"page":{"current":1,"size":10,"ascs":[],"descs":[],"sort":"+id"}}'
# b = json.loads(a)